In [1]:
import sqlite3
import json

# Load JSON Data
with open("mock_data.json", "r") as f:
    data = json.load(f)

# Create DB Connection
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

# Create Tables
cursor.execute("CREATE TABLE teams (team_id INTEGER PRIMARY KEY, team_name TEXT)")
cursor.execute("""CREATE TABLE players (
    player_id INTEGER PRIMARY KEY,
    player_name TEXT,
    team_id INTEGER,
    runs INTEGER,
    wickets INTEGER,
    matches INTEGER,
    playing_role TEXT,
    FOREIGN KEY(team_id) REFERENCES teams(team_id))""")
cursor.execute("""CREATE TABLE matches (
    match_id INTEGER PRIMARY KEY,
    team1_id INTEGER,
    team2_id INTEGER,
    winner_id INTEGER,
    venue TEXT,
    FOREIGN KEY(team1_id) REFERENCES teams(team_id),
    FOREIGN KEY(team2_id) REFERENCES teams(team_id),
    FOREIGN KEY(winner_id) REFERENCES teams(team_id))""")
cursor.execute("""CREATE TABLE innings (
    inning_id INTEGER PRIMARY KEY,
    match_id INTEGER,
    team_id INTEGER,
    runs INTEGER,
    wickets INTEGER,
    FOREIGN KEY(match_id) REFERENCES matches(match_id),
    FOREIGN KEY(team_id) REFERENCES teams(team_id))""")

# Insert Data
cursor.executemany("INSERT INTO teams VALUES (:team_id, :team_name)", data["teams"])
cursor.executemany("INSERT INTO players VALUES (:player_id, :player_name, :team_id, :runs, :wickets, :matches, :playing_role)", data["players"])
cursor.executemany("INSERT INTO matches VALUES (:match_id, :team1_id, :team2_id, :winner_id, :venue)", data["matches"])
cursor.executemany("INSERT INTO innings VALUES (:inning_id, :match_id, :team_id, :runs, :wickets)", data["innings"])
conn.commit()

# =============== 25 SQL Queries =================

queries = {
    1: "SELECT player_name, runs FROM players ORDER BY runs DESC LIMIT 5",
    2: "SELECT player_name, wickets FROM players ORDER BY wickets DESC LIMIT 5",
    3: "SELECT player_name, runs FROM players WHERE runs = 0",
    4: "SELECT team_name, COUNT(*) FROM matches m JOIN teams t ON m.winner_id=t.team_id GROUP BY team_name",
    5: "SELECT player_name, MAX(runs) FROM players",
    6: "SELECT playing_role, COUNT(*) FROM players GROUP BY playing_role",
    7: "SELECT t.team_name, SUM(p.runs) FROM players p JOIN teams t ON p.team_id=t.team_id GROUP BY t.team_name",
    8: "SELECT venue, COUNT(*) FROM matches GROUP BY venue",
    9: "SELECT player_name, AVG(runs) FROM players GROUP BY player_name ORDER BY AVG(runs) DESC LIMIT 3",
    10: "SELECT player_name, runs, wickets FROM players ORDER BY runs DESC, wickets DESC LIMIT 5",
    11: "SELECT match_id, SUM(runs) FROM innings GROUP BY match_id",
    12: "SELECT player_name FROM players WHERE runs > 500",
    13: "SELECT player_name FROM players WHERE wickets > 20",
    14: "SELECT team_name, COUNT(*) FROM players p JOIN teams t ON p.team_id=t.team_id GROUP BY team_name",
    15: "SELECT p.player_name, t.team_name FROM players p JOIN teams t ON p.team_id=t.team_id",
    16: "SELECT team_name, COUNT(*) FROM matches m JOIN teams t ON m.winner_id=t.team_id GROUP BY team_name ORDER BY COUNT(*) DESC LIMIT 1",
    17: "SELECT player_name, matches FROM players ORDER BY matches DESC LIMIT 1",
    18: "SELECT player_name, runs/matches AS avg_runs FROM players ORDER BY avg_runs DESC LIMIT 3",
    19: "SELECT player_name, wickets/matches AS avg_wkts FROM players ORDER BY avg_wkts DESC LIMIT 3",
    20: "SELECT t.team_name, SUM(i.runs) FROM innings i JOIN teams t ON i.team_id=t.team_id GROUP BY t.team_name",
    21: "SELECT match_id, MAX(runs) FROM innings GROUP BY match_id",
    22: "SELECT p.player_name, SUM(p.runs) FROM players p GROUP BY p.player_name HAVING SUM(p.runs)>400",
    23: "SELECT p.player_name, SUM(p.wickets) FROM players p GROUP BY p.player_name HAVING SUM(p.wickets)>10",
    24: "SELECT m.match_id, t.team_name FROM matches m JOIN teams t ON m.winner_id=t.team_id",
    25: "SELECT playing_role, AVG(runs) FROM players GROUP BY playing_role"
}

# Run Queries
for qno, query in queries.items():
    print(f"\n# Query {qno}:")
    for row in cursor.execute(query):
        print(row)


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [2]:
import sqlite3
import json
import pandas as pd

# Step 1: Load JSON data
with open("mock_data.json", "r") as file:
    data = json.load(file)

# Step 2: Create SQLite DB
conn = sqlite3.connect("cricbuzz.db")
cursor = conn.cursor()

# Step 3: Create tables
cursor.execute("DROP TABLE IF EXISTS players")
cursor.execute("DROP TABLE IF EXISTS matches")
cursor.execute("DROP TABLE IF EXISTS scores")

cursor.execute("""
CREATE TABLE players (
    player_id INTEGER PRIMARY KEY,
    name TEXT,
    team TEXT,
    role TEXT,
    runs INTEGER,
    wickets INTEGER
)
""")

cursor.execute("""
CREATE TABLE matches (
    match_id INTEGER PRIMARY KEY,
    team1 TEXT,
    team2 TEXT,
    winner TEXT,
    venue TEXT
)
""")

cursor.execute("""
CREATE TABLE scores (
    score_id INTEGER PRIMARY KEY,
    match_id INTEGER,
    player_id INTEGER,
    runs INTEGER,
    wickets INTEGER,
    FOREIGN KEY (match_id) REFERENCES matches(match_id),
    FOREIGN KEY (player_id) REFERENCES players(player_id)
)
""")

# Step 4: Insert Data
cursor.executemany("INSERT INTO players VALUES (:player_id, :name, :team, :role, :runs, :wickets)", data["players"])
cursor.executemany("INSERT INTO matches VALUES (:match_id, :team1, :team2, :winner, :venue)", data["matches"])
cursor.executemany("INSERT INTO scores VALUES (:score_id, :match_id, :player_id, :runs, :wickets)", data["scores"])

conn.commit()

# Step 5: Define 25 Queries
queries = [
    "SELECT * FROM players",
    "SELECT * FROM matches",
    "SELECT * FROM scores",
    "SELECT name, runs FROM players ORDER BY runs DESC LIMIT 5",
    "SELECT name, wickets FROM players ORDER BY wickets DESC LIMIT 5",
    "SELECT winner, COUNT(*) as wins FROM matches GROUP BY winner",
    "SELECT venue, COUNT(*) as matches_played FROM matches GROUP BY venue",
    "SELECT p.name, SUM(s.runs) as total_runs FROM players p JOIN scores s ON p.player_id = s.player_id GROUP BY p.name ORDER BY total_runs DESC",
    "SELECT p.name, SUM(s.wickets) as total_wickets FROM players p JOIN scores s ON p.player_id = s.player_id GROUP BY p.name ORDER BY total_wickets DESC",
    "SELECT team, COUNT(*) as player_count FROM players GROUP BY team",
    "SELECT m.winner, COUNT(*) as total_wins FROM matches m GROUP BY m.winner ORDER BY total_wins DESC LIMIT 1",
    "SELECT p.team, AVG(p.runs) as avg_runs FROM players p GROUP BY p.team",
    "SELECT p.name, s.runs FROM players p JOIN scores s ON p.player_id = s.player_id WHERE s.runs > 50",
    "SELECT team, SUM(runs) as total_team_runs FROM players GROUP BY team",
    "SELECT team, SUM(wickets) as total_team_wickets FROM players GROUP BY team",
    "SELECT m.venue, COUNT(*) FROM matches m GROUP BY m.venue ORDER BY COUNT(*) DESC LIMIT 1",
    "SELECT p.name FROM players p WHERE p.runs > 10000",
    "SELECT p.name FROM players p WHERE p.wickets > 150",
    "SELECT s.match_id, SUM(s.runs) as total_match_runs FROM scores s GROUP BY s.match_id",
    "SELECT s.match_id, SUM(s.wickets) as total_match_wickets FROM scores s GROUP BY s.match_id",
    "SELECT p.name, p.role FROM players p WHERE p.role = 'Bowler'",
    "SELECT p.name, p.role FROM players p WHERE p.role = 'Batsman'",
    "SELECT team, COUNT(*) as batsmen FROM players WHERE role='Batsman' GROUP BY team",
    "SELECT team, COUNT(*) as bowlers FROM players WHERE role='Bowler' GROUP BY team",
    "SELECT m.match_id, m.winner, m.venue FROM matches m WHERE m.winner='India'"
]

# Step 6: Run Queries
for i, query in enumerate(queries, start=1):
    print(f"\nQuery {i}: {query}")
    df = pd.read_sql_query(query, conn)
    print(df)

conn.close()



Query 1: SELECT * FROM players
   player_id             name         team          role   runs  wickets
0          1      Virat Kohli        India       Batsman  12000        4
1          2     Rohit Sharma        India       Batsman  10000        8
2          3         MS Dhoni        India  Wicketkeeper  10500        1
3          4   Jasprit Bumrah        India        Bowler    500      250
4          5       Babar Azam     Pakistan       Batsman   8000        2
5          6   Shaheen Afridi     Pakistan        Bowler    700      180
6          7  Kane Williamson  New Zealand       Batsman   9000        6
7          8      Trent Boult  New Zealand        Bowler    600      200
8          9      Steve Smith    Australia       Batsman   9500        7
9         10      Pat Cummins    Australia        Bowler    800      210

Query 2: SELECT * FROM matches
   match_id      team1        team2       winner     venue
0         1      India     Pakistan        India     Dubai
1         2    